In [2]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0.2,
    # groq_api_key="add_your_api_key",
    
    model_name="llama-3.3-70b-versatile",
)

In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-36947?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

ModuleNotFoundError: No module named 'langchain_community'